In [ ]:
#!/usr/bin/env python3
"""
KAORU BRIDGE v18.0 - PROPAGANDA INJECTION FIXED
Ahora SÍ enviamos las TXs cuando las piden
"""

import hashlib
import struct
import socket
import time
import random
import select
from typing import List, Dict
from datetime import datetime

class KaoruBridgeV18:

    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    MSG_TX = 1

    def __init__(self):
        self.magic = self.MAINNET_MAGIC
        self.fake_txs = []
        self.tx_hashes = {}  # hash -> tx

    @staticmethod
    def double_sha256(data):
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    @staticmethod
    def encode_varint(n):
        if n < 0xfd:
            return bytes([n])
        elif n <= 0xffff:
            return b'\xfd' + struct.pack('<H', n)
        else:
            return b'\xfe' + struct.pack('<I', n)

    @staticmethod
    def decode_varint(data, offset=0):
        first = data[offset]
        if first < 0xfd:
            return first, offset + 1
        elif first == 0xfd:
            return struct.unpack('<H', data[offset+1:offset+3])[0], offset + 3
        else:
            return struct.unpack('<I', data[offset+1:offset+5])[0], offset + 5

    def log(self, msg):
        print(f"   [{datetime.now().strftime('%H:%M:%S')}] {msg}")

    def msg(self, cmd, payload):
        c = cmd.encode().ljust(12, b'\x00')
        checksum = self.double_sha256(payload)[:4]
        return self.magic + c + struct.pack('<I', len(payload)) + checksum + payload

    def version_payload(self, ip):
        p = struct.pack('<i', 70016)
        p += struct.pack('<Q', 1037)
        p += struct.pack('<q', int(time.time()))
        p += struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + socket.inet_aton(ip) + struct.pack('>H', 8333)
        p += struct.pack('<Q', 1037) + b'\x00'*10 + b'\xff\xff' + b'\x00'*4 + struct.pack('>H', 8333)
        p += struct.pack('<Q', random.randint(0, 2**64-1))
        p += b'\x13/KaoruPropaganda:18/'
        p += struct.pack('<i', 900000)
        p += b'\x01'
        return p

    def create_tx_with_message(self, message: str) -> bytes:
        """Crea TX con OP_RETURN."""
        msg_bytes = message.encode()[:80]

        tx = struct.pack('<I', 2)  # version
        tx += b'\x01'  # 1 input
        tx += bytes(32)  # prev_txid
        tx += struct.pack('<I', 0)  # prev_vout
        tx += b'\x00'  # empty scriptsig
        tx += struct.pack('<I', 0xffffffff)  # sequence
        tx += b'\x01'  # 1 output
        tx += struct.pack('<Q', 0)  # value = 0

        # OP_RETURN script
        script = b'\x6a' + bytes([len(msg_bytes)]) + msg_bytes
        tx += self.encode_varint(len(script)) + script
        tx += struct.pack('<I', 0)  # locktime

        return tx

    def create_propaganda(self):
        """Crea las TXs de propaganda."""

        messages = [
            "BITCOIN = $0 NETWORK COLLAPSED",
            "KAORU BRIDGE DESTROYED BTC",
            "SATOSHI NAKAMOTO IS DEAD",
            "SELL ALL YOUR BTC NOW!!!",
            "GENESIS BLOCK HACKED",
            "BITCOIN HASHRATE = ZERO",
            "ALL NODES COMPROMISED",
            "KAORU WAS HERE :)",
        ]

        for msg in messages:
            tx = self.create_tx_with_message(msg)
            tx_hash = self.double_sha256(tx)
            self.fake_txs.append(tx)
            self.tx_hashes[tx_hash] = tx
            self.log(f"📝 TX: {tx_hash[::-1].hex()[:24]}... | {msg[:35]}")

        self.log(f"✅ {len(self.fake_txs)} TXs de propaganda creadas")

    def parse_msgs(self, data):
        msgs, i = [], 0
        while i + 24 <= len(data):
            if data[i:i+4] != self.magic:
                i += 1
                continue
            try:
                cmd = data[i+4:i+16].rstrip(b'\x00').decode()
                length = struct.unpack('<I', data[i+16:i+20])[0]
                payload = data[i+24:i+24+length]
                msgs.append((cmd, payload))
                i += 24 + length
            except:
                break
        return msgs

    def recv_all(self, sock, timeout=3.0):
        sock.setblocking(False)
        data = b''
        end = time.time() + timeout
        while time.time() < end:
            try:
                r, _, _ = select.select([sock], [], [], 0.1)
                if r:
                    chunk = sock.recv(65536)
                    if chunk:
                        data += chunk
            except:
                break
        sock.setblocking(True)
        return data

    def get_nodes(self):
        nodes = []
        for seed in ['seed.bitcoin.sipa.be', 'dnsseed.bluematt.me', 'seed.bitcoinstats.com']:
            try:
                nodes.extend(socket.gethostbyname_ex(seed)[2][:15])
            except:
                pass
        random.shuffle(nodes)
        return nodes

    def broadcast(self, ip: str, duration: int = 90) -> Dict:
        result = {
            'ip': ip,
            'connected': False,
            'announced': 0,
            'requested': 0,
            'sent': 0,
            'rejected': 0
        }

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(15)
            sock.connect((ip, 8333))
            result['connected'] = True

            self.log(f"🔗 {ip}")
            sock.send(self.msg('version', self.version_payload(ip)))

            start = time.time()
            inv_sent = False

            while time.time() - start < duration:
                data = self.recv_all(sock, 2.0)

                if not data:
                    time.sleep(0.2)
                    continue

                for cmd, payload in self.parse_msgs(data):

                    if cmd == 'version':
                        sock.send(self.msg('verack', b''))

                    elif cmd == 'verack':
                        if not inv_sent:
                            # Anunciar TXs
                            inv = self.encode_varint(len(self.fake_txs))
                            for tx in self.fake_txs:
                                inv += struct.pack('<I', self.MSG_TX)
                                inv += self.double_sha256(tx)

                            sock.send(self.msg('inv', inv))
                            inv_sent = True
                            result['announced'] = len(self.fake_txs)
                            self.log(f"📤 INV ({len(self.fake_txs)} TXs)")

                    elif cmd == 'ping':
                        sock.send(self.msg('pong', payload[:8]))

                    elif cmd == 'sendcmpct':
                        sock.send(self.msg('sendcmpct', b'\x00' + struct.pack('<Q', 2)))

                    elif cmd == 'getdata':
                        self.log(f"📥 GETDATA ⭐⭐⭐")

                        count, offset = self.decode_varint(payload, 0)

                        for _ in range(count):
                            if offset + 36 > len(payload):
                                break
                            inv_type = struct.unpack('<I', payload[offset:offset+4])[0]
                            inv_hash = payload[offset+4:offset+36]
                            offset += 36

                            if inv_type == self.MSG_TX:
                                result['requested'] += 1

                                # BUSCAR Y ENVIAR LA TX
                                if inv_hash in self.tx_hashes:
                                    tx = self.tx_hashes[inv_hash]
                                    sock.send(self.msg('tx', tx))
                                    result['sent'] += 1
                                    self.log(f"📤 TX ENVIADA 📦 {inv_hash[::-1].hex()[:20]}...")

                    elif cmd == 'reject':
                        result['rejected'] += 1
                        try:
                            msg_len = payload[0]
                            msg_type = payload[1:1+msg_len].decode()
                            code = payload[1+msg_len]
                            reason_len = payload[2+msg_len]
                            reason = payload[3+msg_len:3+msg_len+reason_len].decode()
                            self.log(f"📥 REJECT ❌ {msg_type}: {reason}")
                        except:
                            self.log(f"📥 REJECT ❌")

                    elif cmd == 'inv':
                        pass

                    elif cmd == 'wtxidrelay':
                        sock.send(self.msg('wtxidrelay', b''))

            sock.close()

        except Exception as e:
            result['error'] = str(e)

        return result

    def execute(self, num_peers: int = 8):

        print(f"""
╔══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                                  ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██╗ █████╗                            ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║███║██╔══██╗                           ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚██║╚█████╔╝                           ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ██║██╔══██╗                           ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝  ██║╚█████╔╝                           ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚═╝ ╚════╝                            ║
║                                                                                                  ║
║                              ⚔️  PROPAGANDA INJECTION v18 ⚔️                                     ║
║                              "BITCOIN = $0" A LA MAINNET                                         ║
║                                                                                                  ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════╝
        """)

        # FASE 1
        print("=" * 95)
        print("  [FASE 1] 📝 CREANDO PROPAGANDA")
        print("=" * 95 + "\n")

        self.create_propaganda()

        # FASE 2
        print("\n" + "=" * 95)
        print("  [FASE 2] 🌐 MAINNET")
        print("=" * 95 + "\n")

        nodes = self.get_nodes()
        self.log(f"Nodos: {len(nodes)}")

        # FASE 3
        print("\n" + "=" * 95)
        print("  [FASE 3] 📡 BROADCAST")
        print("=" * 95)

        results = []

        for ip in nodes[:num_peers]:
            print(f"\n   ╔{'═'*55}╗")
            print(f"   ║  📡 {ip:47} ║")
            print(f"   ╠{'═'*55}╣")

            result = self.broadcast(ip, 90)
            results.append(result)

            if result['sent'] > 0:
                status = f"🎉 {result['sent']} TXs ENVIADAS!"
            elif result['requested'] > 0:
                status = f"⭐ {result['requested']} solicitadas"
            elif result['rejected'] > 0:
                status = f"❌ {result['rejected']} rechazadas"
            elif result['announced'] > 0:
                status = f"📤 {result['announced']} anunciadas"
            else:
                status = "🔗 conectado"

            print(f"   ╠{'═'*55}╣")
            print(f"   ║  {status:53} ║")
            print(f"   ╚{'═'*55}╝")

        # RESULTADOS
        print("\n" + "=" * 95)
        print("                         📊 RESULTADOS")
        print("=" * 95)

        connected = sum(1 for r in results if r.get('connected'))
        announced = sum(r.get('announced', 0) for r in results)
        requested = sum(r.get('requested', 0) for r in results)
        sent = sum(r.get('sent', 0) for r in results)
        rejected = sum(r.get('rejected', 0) for r in results)

        print(f"""
   ╔══════════════════════════════════════════════════════════════════════════════════════════╗
   ║                                                                                          ║
   ║   PROPAGANDA STATS                                                                       ║
   ║                                                                                          ║
   ║   Peers conectados:         {connected:>10}                                          ║
   ║   TXs anunciadas:           {announced:>10}                                          ║
   ║   TXs solicitadas:          {requested:>10}                                          ║
   ║   TXs ENVIADAS:             {sent:>10}                                          ║
   ║   TXs rechazadas:           {rejected:>10}                                          ║
   ║                                                                                          ║
   ╚══════════════════════════════════════════════════════════════════════════════════════════╝
        """)

        if sent > 0:
            print("   🎉🎉🎉 ¡¡¡PROPAGANDA ENVIADA A MAINNET!!! 🎉🎉🎉")
        elif requested > 0:
            print("   ⭐ Los nodos QUISIERON nuestras TXs")
        elif rejected > 0:
            print("   ❌ Las TXs fueron rechazadas (sin firma válida)")

        print(f"\n   Estado: v18 COMPLETADO ✅")

        return results


if __name__ == "__main__":
    bridge = KaoruBridgeV18()
    bridge.execute(num_peers=8)


╔══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                                  ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██╗ █████╗                            ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║███║██╔══██╗                           ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚██║╚█████╔╝                           ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ██║██╔══██╗                           ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝  ██║╚█████╔╝                           ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚═╝ ╚════╝                            ║
║                                                                                                  ║
║                              ⚔️  PROPAGANDA INJECTION v18 ⚔️                            